<a href="https://colab.research.google.com/github/11o9/A/blob/master/Parking_World_RL_MVP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import C# Module

In [0]:
import sys
import clr
import os

cwd = os.getcwd()

sys.path.append(cwd+r"C:\..\ParkingWorldMVP\bin\Debug")

import clr

clr.AddReference(r"clipper_library")
clr.AddReference(r"Comar")
clr.AddReference(r"ParkingWorldMVP")
clr.AddReference(r"Rhino3dmIO")

from ParkingWorld import *

In [0]:
env = PWEnvironment()

In [0]:
import numpy as np
import tensorflow as tf


In [0]:
def one_hot(x):
    return np.identity(16)[x:x+1]

In [0]:
input_size = env.NUM_STATE_FEATURE
output_size = env.NUM_ACTION
learning_rate = 0.1


In [0]:
X = tf.placeholder(shape = [1,input_size], dtype = tf.float32)
W = tf.Variable(tf.random_uniform([input_size, output_size],0,0.01))
Qpred = tf.matmul(X,W)


In [0]:
Y = tf.placeholder(shape = [1, output_size], dtype = tf.float32)

loss = tf.reduce_sum(tf.square(Y-Qpred))

train = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

dis = .99
num_episodes = 1000
rList = []

sess = tf.Session()

## START LEARNING

In [0]:
sess.run(tf.global_variables_initializer())

#print(sess.run(W))

#Start Episode
for i in range(num_episodes):

    #print("episode " ,i)

    #Starting State that is always zero
    s = env.Reset()
    #print("start state : ",s)

    #Epsillon
    e = 1. / ((i/50)+10)
    rAll = 0
    done = False
    local_loss = []

    while not done:

        Qs =sess.run(Qpred,feed_dict={X : one_hot(s)})
        #print("Start Qs : " ,Qs)
        if np.random.rand(1) <e:
            a = env.action_space.sample()
        else:
            a = np.argmax(Qs)

        #print("selected action : ",a)
        s1,reward,done,_ = env.step(a)
        #print("next sate: ",s1 ," | reward : ", reward, " | done : ", done)


        if done:
            Qs[0,a] =reward
            #print(Qs)
        else:    
            Qs1 = sess.run(Qpred, feed_dict = { X : one_hot(s1)})
            Qs[0,a] = reward + dis * np.max(Qs1)

        #print("End Qs : " ,Qs)

        sess.run(train, feed_dict = {X:one_hot(s),Y:Qs})

        rAll += reward
        s =s1

        #print("")

    rList.append(rAll)